In [2]:
from settings import load_settings
setting = load_settings()

Loading .env file


In [1]:
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
web_loader = WebBaseLoader("https://weather.com/weather/tenday/l/1a8af5b9d8971c46dd5a52547f9221e22cd895d8d8639267a87df614d0912830")

In [3]:
docs: list[Document] = web_loader.load()

In [4]:
docs

[Document(metadata={'source': 'https://weather.com/weather/tenday/l/1a8af5b9d8971c46dd5a52547f9221e22cd895d8d8639267a87df614d0912830', 'title': '10-Day Weather Forecast for Paris, France - The Weather Channel | weather.com', 'description': 'Be prepared with the most accurate 10-day forecast for Paris, France with highs, lows, chance of precipitation from The Weather Channel and Weather.com', 'language': 'en-US'}, page_content="\n\n\n10-Day Weather Forecast for Paris, France - The Weather Channel | weather.com\n\n\n\n\n\n\n\nAdvertisementAdvertisementSkip to Main ContentAccessibility HelpThe Weather ChannelType at least three characters to start auto complete. Recently searched locations will be displayed if there is no search query. The first option will be automatically selected. Use up and down arrows to change selection. Use escape to clear.Search City or Zip CodeSearchrecentsClear AllYou have no recent locationsGlobeUS°FArrow down°F°CHybridImperial - F / mph / miles / inchesAmerica

In [5]:
for doc in docs:
    print(doc.page_content)




10-Day Weather Forecast for Paris, France - The Weather Channel | weather.com







AdvertisementAdvertisementSkip to Main ContentAccessibility HelpThe Weather ChannelType at least three characters to start auto complete. Recently searched locations will be displayed if there is no search query. The first option will be automatically selected. Use up and down arrows to change selection. Use escape to clear.Search City or Zip CodeSearchrecentsClear AllYou have no recent locationsGlobeUS°FArrow down°F°CHybridImperial - F / mph / miles / inchesAmericasArrow DownAntigua and Barbuda | EnglishArgentina | EspañolBahamas | EnglishBarbados | EnglishBelize | EnglishBolivia | EspañolBrazil | PortuguêsCanada | EnglishCanada | FrançaisChile | EspañolColombia | EspañolCosta Rica | EspañolDominica | EnglishDominican Republic | EspañolEcuador | EspañolEl Salvador | EspañolGrenada | EnglishGuatemala | EspañolGuyana | EnglishHaiti | FrançaisHonduras | EspañolJamaica | EnglishMexico | EspañolNicaragu

In [6]:
len(docs)

1

In [7]:
text = ""
for doc in docs:
    text = text + doc.page_content
text

"\n\n\n10-Day Weather Forecast for Paris, France - The Weather Channel | weather.com\n\n\n\n\n\n\n\nAdvertisementAdvertisementSkip to Main ContentAccessibility HelpThe Weather ChannelType at least three characters to start auto complete. Recently searched locations will be displayed if there is no search query. The first option will be automatically selected. Use up and down arrows to change selection. Use escape to clear.Search City or Zip CodeSearchrecentsClear AllYou have no recent locationsGlobeUS°FArrow down°F°CHybridImperial - F / mph / miles / inchesAmericasArrow DownAntigua and Barbuda | EnglishArgentina | EspañolBahamas | EnglishBarbados | EnglishBelize | EnglishBolivia | EspañolBrazil | PortuguêsCanada | EnglishCanada | FrançaisChile | EspañolColombia | EspañolCosta Rica | EspañolDominica | EnglishDominican Republic | EspañolEcuador | EspañolEl Salvador | EspañolGrenada | EnglishGuatemala | EspañolGuyana | EnglishHaiti | FrançaisHonduras | EspañolJamaica | EnglishMexico | Esp

In [8]:
with open("text.txt", "w+") as fh:
    fh.write(text)

In [10]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import Field
# Define your desired data structure.
from pydantic import BaseModel, Field

class DayTemperature(BaseModel):
    date: str = Field("date of measurements")
    day: str = Field("day temperature")
    night: str = Field("night temperature")
    weather_conditions: str = Field("weather conditions for day and night")
    chance_of_rain: str = Field("chance of rain (day and night)")
    wind: str = Field("wind speed and direction (day and night)")
    humidity: str = Field("humidity (day and night)")
    UV: str = Field(" UV index")
    sunrise_sunset: str = Field("sunrise and sunset times")



In [11]:
parser = JsonOutputParser(pydantic_object=DayTemperature)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"date": {"default": "date of measurements", "title": "Date", "type": "string"}, "day": {"default": "day temperature", "title": "Day", "type": "string"}, "night": {"default": "night temperature", "title": "Night", "type": "string"}, "weather_conditions": {"default": "weather conditions for day and night", "title": "Weather Conditions", "type": "string"}, "chance_of_rain": {"default": "chance of rain (day and night)", "title": "Chance Of Rain", "type": "string"}, "wind": {"default": "wind speed and direction (day and nigh

In [12]:
prompt = PromptTemplate(
    template=
    """
        Please provide the 10-day weather forecast for Paris, France, in a structured table format. Include both day and night details for each day. 
        Response should include the following fields:
        {format_instructions}
    Please ensure the data is well-organized and easy to read.
    <content>
    {content}
    </conetnt>
    """,
    input_variables=["content"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [13]:
from langchain_mistralai import ChatMistralAI
from settings import load_settings
settings = load_settings()

model = ChatMistralAI(api_key=settings.mistral_api_key, model_name="mistral-large-latest")


Loading .env file


In [65]:
chain = prompt | model | parser
output = chain.invoke({
    "content": text
})

In [66]:
import pandas as pd

df = pd.DataFrame(output)

In [69]:
df.head()

,date,day,night,weather_conditions,chance_of_rain,wind,humidity,UV,sunrise_sunset
0,Mon 07,Heavy Rain 56°,Heavy Rain 56°,Heavy Rain,100%,SSW 9 mph,92%,0 of 11,"Sunrise: 8:00 am, Sunset: 7:14 pm"
1,Tue 08,Partly Cloudy 67°,Scattered Showers 57°,"AM Clouds/PM Sun, Cloudy with periods of light...","24% (day), 68% (night)",SSW 13 mph,"79% (day), 86% (night)","3 of 11 (day), 0 of 11 (night)","Sunrise: 8:02 am, Sunset: 7:12 pm"
2,Wed 09,Rain 61°,Rain 55°,Rain,"98% (day), 100% (night)","SSW 9 mph (day), NW 11 mph (night)","93% (day), 95% (night)","1 of 11 (day), 0 of 11 (night)","Sunrise: 8:03 am, Sunset: 7:10 pm"
3,Thu 10,Scattered Showers 58°,Scattered Showers 46°,Steady light rain in the morning. Showers cont...,"59% (day), 41% (night)","WNW 9 mph (day), NW 6 mph (night)","84% (day), 88% (night)","2 of 11 (day), 0 of 11 (night)","Sunrise: 8:05 am, Sunset: 7:08 pm"
4,Fri 11,Partly Cloudy 56°,Partly Cloudy 43°,Partly cloudy,"17% (day), 6% (night)","NE 4 mph (day), E 5 mph (night)","77% (day), 80% (night)","3 of 11 (day), 0 of 11 (night)","Sunrise: 8:06 am, Sunset: 7:06 pm"
